# Part 1: Forward Mode Automatic Differentiation

Forward mode AD can simply be implemented by defining a class to represent [dual numbers](https://en.wikipedia.org/wiki/Dual_number) which hold the value and its derivative. The following skeleton defines a dual number and implements multiplication. 

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [39]:
import math

class DualNumber:
    def __init__(self, value, dvalue):
        self.value = value
        self.dvalue = dvalue

    def __str__(self):
        return str(self.value) + " + " + str(self.dvalue) + "ε"
    
    def __add__(self, other):
        return DualNumber(self.value + other.value,self.dvalue + other.dvalue)

    
    def __sub__(self, other):
        return DualNumber(self.value - other.value,self.dvalue - other.dvalue)

        raise NotImplementedError()

    def __mul__(self, other):
        return DualNumber(self.value * other.value,self.dvalue * other.value + other.dvalue * self.value)
    
    def __truediv__(self, other):
        return DualNumber(self.value / other.value,(self.dvalue * other.value - other.dvalue * self.value) / other.value ** 2)
        
    def __pow__(self, other):
        return DualNumber(self.value ** other.value, other.value * self.value ** (other.value-1) * other.dvalue)

In [55]:
# Tests
print(DualNumber(1,0)) 
print(DualNumber(1,0).value) 
print(DualNumber(1,0).dvalue) 

x = DualNumber(0.5,1)
y = DualNumber(4.2,1)
z = x.__add__(y)
print(z)
z = x.__mul__(y)
print(z)
z = x.__mul__(y) + sin(x)
print(z)
print(DualNumber(1,0) + DualNumber(1,0))
# print(DualNumber(1,0) / DualNumber(1,0))
# print(DualNumber(1,0) - DualNumber(1,0))
# print(DualNumber(1,0) ** DualNumber(1,0))

1 + 0ε
1
0
4.7 + 2ε
2.1 + 4.7ε
2.579425538604203 + 5.577582561890373ε
2 + 0ε


## Implementing math functions

We also need to implement some core math functions. Here's the sine function for a dual number:

In [47]:
def sin(x):
    return DualNumber(math.sin(x.value), math.cos(x.value)*x.dvalue)

__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [48]:
# TODO: implement additional math functions on dual numbers

def cos(x):
    # YOUR CODE HERE
    return DualNumber(math.cos(x.value), -math.sin(x.value)*x.dvalue)
    raise NotImplementedError()

def tan(x):
    # YOUR CODE HERE
    return DualNumber(math.tan(x.value), x.dvalue/math.cos(x.value)**2)
    raise NotImplementedError()

def exp(x):
    # YOUR CODE HERE
    return DualNumber(math.exp(x.value), math.exp(x.value)*x.dvalue)
    raise NotImplementedError()

In [49]:
# Tests
assert cos(DualNumber(0,0)).value == 1
assert tan(DualNumber(0,0)).value == 0
assert exp(DualNumber(0,0)).value == 1

1.0 + -0.0ε


## Time to try it out

We're now in a position to try our implementation.

__Task:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 

In [63]:
x = DualNumber(0.5,1)
y = DualNumber(4.2,0)
z = x.__mul__(y) + sin(x)
print(z)
print(z.value)
print(z.dvalue)

2.579425538604203 + 5.077582561890373ε
2.579425538604203
5.077582561890373


__Task__: Differentiate the above function with respect to $x$ and write the symbolic derivatives in the following box. Verify the result computed above is correct by plugging-in the values into your symbolic gradient expression.

In [65]:
dzdx = y.value + math.cos(x.value)
print(dzdx)

5.077582561890373


__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above) and store the derivative in the variable `dzdy` (just the derivative, not the Dual Number). Verify by hand that the result is correct.

In [66]:
x = DualNumber(0.5,0)
y = DualNumber(4.2,1)
dzdy1 = x.value
print(dzdy1)
z = x.__mul__(y) + sin(x)
dzdy2 = z.dvalue
print(dzdy2)

0.5
0.5


# Part 2: Reverse Mode Automatic Differentiation

Dynamic Reverse mode AD can be implemented by declaring a class to represent a value and the child expressions that the value depends on. We've provided the implementation that was shown in the lecture slides as a basis below, but it's missing some parts that will make it useful.

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [104]:
import math

class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None

    def grad(self):
        if self.grad_value is None:
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value
    
    def __str__(self):
        return str(self.value)

    def __mul__(self, other):
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z

    def __add__(self, other):
        z = Var(self.value+other.value)
        self.children.append((1,z))
        other.children.append((1,z))
        return z
        
    def __truediv__(self, other):
        z = Var(self.value / other.value)
        self.children.append((1/other.value, z))
        other.children.append((-self.value/other.value**2, z))
        return z
    
    def __sub__(self, other):
        z = Var(self.value - other.value)
        self.children.append((1, z))
        other.children.append((1, z))
        return z
    
    def __pow__(self, other):
        z = Var(self.value ** other.value)
        self.children.append((other.value*(x**(other.value-1)), z))
        other.children.append((self.value ** other.value*math.log(self.value), z))
        return z

#     raise NotImplementedError()

In [105]:
x = Var(0.5)
print(x.value)
print(x.children)
print(x.grad)
y = Var(4.2)
print(y.value)
print(y.children)
print(y.grad)
z = x.__add__(y)
print(z)
print(z.value)
print(z.grad)

0.5
[]
<bound method Var.grad of <__main__.Var object at 0x000002E48471D430>>
4.2
[]
<bound method Var.grad of <__main__.Var object at 0x000002E48471DC10>>
4.7
4.7
<bound method Var.grad of <__main__.Var object at 0x000002E484813F40>>


## Implementing math functions

Just like when we were looking at Forward Mode AD, we also need to implement some core math functions. Here's the sine function for a `Var`:

In [74]:
def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [75]:
# TODO: implement additional math functions on dual numbers

def cos(x):
    z = Var(math.cos(x.value))
    x.children.append((-math.sin(x.value), z))
    return z
    raise NotImplementedError()

def tan(x):
    z = Var(math.tan(x.value))
    x.children.append((1/(math.cos(x.value))**2, z))
    return z
    raise NotImplementedError()

def exp(x):
    z = Var(math.exp(x.value))
    x.children.append((math.exp(x.value), z))
    return z
    raise NotImplementedError()

In [83]:
# Tests
assert cos(Var(0)).value == 1
assert tan(Var(0)).value == 0
assert exp(Var(0)).value == 1

0


## Time to try it out

We're now in a position to try our implementation.

__Tasks:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 
- Verify that the result is correct by hand-differentiating the function.

In [106]:
x = Var(0.5)
y = Var(4.2)
z = x * y + sin(x)
print('z:', z)

z.grad_value = 1.0 #Note that we have to 'seed' the gradient of z to 1 (e.g. ∂z/∂z=1) before computing grads
print('∂z/∂x:',x.grad())

z: 2.579425538604203
∂z/∂x: 5.077582561890373


In [107]:
x = Var(0.5)
y = Var(4.2)
z = x.__mul__(y) + sin(x)
print('z:', z)

z.grad_value = 1.0 #Note that we have to 'seed' the gradient of z to 1 (e.g. ∂z/∂z=1) before computing grads
print('∂z/∂x:',x.grad())

z: 2.579425538604203
∂z/∂x: 5.077582561890373


__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above). Store the resultant gradient in the variable `dzdy`. Verify by hand that the result is correct.

In [112]:
x = Var(0.5)
y = Var(4.2)
z = y.__mul__(x) + sin(x)
z.grad_value = 1.0
dzdy = y.grad()
print('∂z/∂y:', dzdy)

∂z/∂y: 0.5


In [113]:
x = Var(0.5)
y = Var(4.2)
z = y*x + sin(x)
z.grad_value = 1.0
dzdy = y.grad()
print('∂z/∂y:', dzdy)

∂z/∂y: 0.5


## Differentiating Algorithms

Now, let's look at doing something wacky: differentiate an algorithm. For this example, we'll use an algorithm that is in a sense static (in this particular case the upper limit of the for loop is predetermined). However, it is not difficult to see that AD is much more general, and could even be applied to stochastic algorithms (say if we replaced the upper limit of the loop below with `Math.floor(Math.random() * 10)` for example).

__Task:__ Consider the following algorithm and in the box below it manually compute the value of $z$ and the gradient $\partial z/\partial x$ at the end of execution.

In [130]:
x = Var(0.5)
z = Var(1)
for i in range(0,2):
    z = (z + Var(i)) * x * x

z1 = (z0+Var(0)) * 0.5 * 0.5 = (1+0) * 0.25 = 0.25

z2 = (z1+Var(1)) * 0.5 * 0.5 = (0.25+1) * 0.25 = 0.3125

__Task__: Now use the code block below to print out the gradient computed by our reverse AD by storing the result in a variable called `grad`. Does it match?

In [131]:
print(z.value)
z.grad_value = 1.0
print(x.grad())

0.3125
1.5


# Part 3: Reverse Mode Automatic Differentiation with PyTorch

In [132]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

PyTorch implements Dynamic Reverse Mode Automatic Differentiation, much like we did in the previous exercise. There is one really major difference in what PyTorch provides over our simple example: it works directly with matrices (`Tensor`s) rather than with scalars (although obviously a matrix can represent a scalar).

In this tutorial, we'll explore PyTorch's AD implementation. Note that we're using the API of PyTorch 0.4 or later which simplifies use of AD (previous versions required wrapping all `Tensor`s that you wanted to compute gradients of in `Variable` objects; PyTorch 0.4 removes the need to do this and allows `Tensor`s themselves to track gradients).

We'll start with the simple example we tried earlier in the code block below:

__Task:__ Run the following code and verify the solution is correct

In [133]:
import torch

# set up the problem
x = torch.tensor(0.5, requires_grad=True)
y = torch.tensor(4.2, requires_grad=True)
z = x * y + torch.sin(x)

print("z = " + str(z.item()))

z.backward() # this goes through the computation graph and accumulates the gradients in the cached .grad attributes
print("dz/dx = " + str(x.grad.item()))
print("dz/dy = " + str(y.grad.item()))

z = 2.57942533493042
dz/dx = 5.077582359313965
dz/dy = 0.5


As with our own AD implementation, PyTorch lets us differentiate through an algorithm.

__Task__: Use the block below to compute the gradient $\partial z/\partial x$ of the following pseudocode algorithm and store the result in the `dzdx` variable:

    x = 0.5
    z = 1
    i = 0
    while i<2:
        z = (z + i) * x * x
        i = i + 1

In [144]:
x = torch.tensor(0.5, requires_grad=True)
z = torch.tensor(1.0, requires_grad=True)

for i in range(0,2):
    z = (z + i) * x * x
    
z.backward()

dzdx = x.grad.item()

print(dzdx)

# # YOUR CODE HERE
# raise NotImplementedError()

1.5


In [145]:
assert dzdx


## PyTorch limitations: in-place operations and aliasing

PyTorch will throw an error at runtime if you try to differentiate through an in-place operation on a tensor. 

__Task__: Run the following code to see this in action.

In [146]:
x = torch.tensor(1.0, requires_grad=True)
y = x.tanh()
y.add_(3) # inplace addition
y.backward()

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of TanhBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

Aliasing is also something that can't be differentiated through and will result in a slightly more cryptic error.

__Task__: Run the following code to see this in action. If you don't understand what this code does add some `print` statements to show the values of `x` and `y` at various points.

In [148]:
x = torch.tensor([1, 2, 3, 4], requires_grad=True, dtype=torch.float)
y = x[:1]
print(y)
y.add_(3)
y.backward()

tensor([1.], grad_fn=<SliceBackward0>)


RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

## Dealing with multiple outputs

PyTorch can deal with the case where there are multiple output variables if we can formulate the expression in terms of tensor operations. Consider the example from the presentation for example:

$$\begin{cases}
     z = 2x + \sin x\\
     v = 4x + \cos x
\end{cases}$$

We could formulate this as:

$$
\begin{bmatrix}z \\ v\end{bmatrix} = \begin{bmatrix}2 \\ 4\end{bmatrix} \odot \bar{x} + \begin{bmatrix}1 \\ 0\end{bmatrix} \odot \sin\bar x + \begin{bmatrix}0 \\ 1\end{bmatrix} \odot \cos\bar x
$$

where 

$$
\bar x = \begin{bmatrix}x \\ x\end{bmatrix}
$$

and $\odot$ represents the Hadamard or element-wise product. This is demonstrated using PyTorch in the following code block.

__Task:__ run the code below.

In [149]:
x = torch.tensor([[1.0],[1.0]], requires_grad=True)

zv = ( torch.tensor([[2.0],[4.0]]) * x +
         torch.tensor([[1.0], [0.0]]) * torch.sin(x) +
         torch.tensor([[0.0], [1.0]]) * torch.cos(x) )
        
zv.backward(torch.tensor([[1.0],[1.0]])) # Note as we have "multiple outputs" we must pass in a tensor of weights of the correct size

print(x.grad)

tensor([[2.5403],
        [3.1585]])


__Task:__ Use the following box to write down the derivative of the expression for $\begin{bmatrix}z \\ v\end{bmatrix}$ and verify the gradients $\partial z/\partial x$ and $\partial v/\partial x$ are correct for $x=1$.

$\partial z/\partial x$ = 2 + cos(x)

$\partial v/\partial x$ = 4 - sin(x)

when x = 1,

$\partial z/\partial x$ = 2 + cos(x) = 2 + cos(1) = 2.5

$\partial v/\partial x$ = 4 - sin(x) = 4 - sin(1) = 3.1

## Gradient descent & gradients with respect to a vector
Let's put everything together and using automatically computed gradients to find the minima of a function by taking steps down the gradient from an initial position. Rather than explicitly creating each input variable as a scalar as in the previous examples, we'll use a vector instead (so our gradients will be with respect to each element of the vector).

__Task:__ work through the following example to see how taking gradients with respect to a vector works & how simple gradient descent optimisation can be implemented.

In [153]:
# This is our starting vector
initial=[[2.0], [1.0], [10.0]]

# This is the function we will optimise (feel free to work out the analytic minima!)
def function(x):
    return x[0]**2 + x[1]**2 + x[2]**2

x = torch.tensor(initial, requires_grad=True, dtype=torch.float)
# print(x)
# print(x.grad)
# print(x.data)
for i in range(0,100):
    # manually dispose of the gradient (in reality it would be better to detach and zero it to reuse memory)
    x.grad=None
    # evaluate the function
    J = function(x) 
    # auto-compute the gradients at the previously evaluated point x
    J.backward()
    # compute the update
    x.data = x - x.grad*0.1 
    
    if i%10 == 0:
        print((x, function(x).item()))

(tensor([[1.6000],
        [0.8000],
        [8.0000]], requires_grad=True), 67.19999694824219)
(tensor([[0.1718],
        [0.0859],
        [0.8590]], requires_grad=True), 0.7747630476951599)
(tensor([[0.0184],
        [0.0092],
        [0.0922]], requires_grad=True), 0.008932411670684814)
(tensor([[0.0020],
        [0.0010],
        [0.0099]], requires_grad=True), 0.00010298370034433901)
(tensor([[0.0002],
        [0.0001],
        [0.0011]], requires_grad=True), 1.1873213452417986e-06)
(tensor([[2.2836e-05],
        [1.1418e-05],
        [1.1418e-04]], requires_grad=True), 1.3688886468798955e-08)
(tensor([[2.4520e-06],
        [1.2260e-06],
        [1.2260e-05]], requires_grad=True), 1.5782215811999123e-10)
(tensor([[2.6328e-07],
        [1.3164e-07],
        [1.3164e-06]], requires_grad=True), 1.8195657654207498e-12)
(tensor([[2.8270e-08],
        [1.4135e-08],
        [1.4135e-07]], requires_grad=True), 2.0978168543115717e-14)
(tensor([[3.0354e-09],
        [1.5177e-09],
        [

__Task__: Answer the following question in the box below: Why must the update in the code above be assigned to `x.data` rather than `x`?

## Differentiating through random operations

We'll end with an example that will be important later in the course: differentiating with respect to the parameters of a random number generator.

Assume that as some part of a differentiable program that we write we wish to incorporate a random element where we sample values, $z$ from a Normal distribution: $z \sim \mathcal{N}(\mu,\sigma^2)$. We want to learn the parameters of the generator $\mu$ and $\sigma^2$, but how can we do this? In a differentiable program setting we want to differentiate with respect to these parameters, but at first glance it isn't at all obvious what this means as the generator _just_ produces numbers which themselves don't have gradients.

The answer is often called the _reparameterisation trick_: If we note that sampling a Normal distribution with a specfic mean and variance is equivalent to drawing numbers from a standard Normal distribution and scaling and shifting them: $z \sim \mathcal{N}(\mu,\sigma^2) \equiv z \sim \mu + \sigma\mathcal{N}(0,1)\equiv z = \mu + \sigma \zeta\, \rm{where}\, \zeta\sim\mathcal{N}(0,1)$. With this reparameterisation the gradients with respect to the parameters are obvious.

The following code block demonstrates this in practice; each of the gradients can be interpreted as how much an infintesimal change in $\mu$ or $\sigma$ would change $z$ if we could repeat the sampling operation again with the same value of `torch.randn(1)` being produced:

In [154]:
mu = torch.tensor(1.0, requires_grad=True)
sigma = torch.tensor(1.0, requires_grad=True)

for i in range(10):
    mu.grad = None
    sigma.grad = None
    
    z = mu + sigma * torch.randn(1)
    z.backward()
    print("z:", z.item(), "\tdzdmu:", mu.grad.item(), "\tdzdsigma:", sigma.grad.item())

z: 0.23901230096817017 	dzdmu: 1.0 	dzdsigma: -0.7609876990318298
z: -0.2584792375564575 	dzdmu: 1.0 	dzdsigma: -1.2584792375564575
z: 3.4020943641662598 	dzdmu: 1.0 	dzdsigma: 2.4020943641662598
z: 2.4265389442443848 	dzdmu: 1.0 	dzdsigma: 1.4265388250350952
z: 2.4836654663085938 	dzdmu: 1.0 	dzdsigma: 1.4836653470993042
z: 0.2979324460029602 	dzdmu: 1.0 	dzdsigma: -0.7020675539970398
z: -0.3962228298187256 	dzdmu: 1.0 	dzdsigma: -1.3962228298187256
z: 0.05982029438018799 	dzdmu: 1.0 	dzdsigma: -0.940179705619812
z: 0.9708083271980286 	dzdmu: 1.0 	dzdsigma: -0.029191698879003525
z: 1.7798521518707275 	dzdmu: 1.0 	dzdsigma: 0.7798521518707275
